<a href="https://colab.research.google.com/github/pedroigp/Fase3-Techchallenge/blob/main/TC3_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tech Challenge 3**

Para realizar o projeto proposto, vamos utilizar o modelo BERT (Bidirectional Encoder Representations from Transformers) da Hugging Face, que é um dos modelos mais populares para tarefas de processamento de linguagem natural (NLP). Ele é gratuito, de código aberto e altamente eficaz para tarefas como fine-tuning e geração de respostas contextualizadas.

Vamos dividir o processo em etapas e fornecer o código para cada uma delas:

1. Instalação das Bibliotecas Necessárias

In [ ]:
pip install transformers datasets pandas torch faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Vamos carregar e preparar o dataset AmazonTitles-1.3MM - Utilizando versão com 50 mil registros.

In [ ]:
import pandas as pd
import numpy as np

# Carregar o dataset
dataset_path = "/content/drive/MyDrive/FIAP/trn_50k.json"
df = pd.read_json(dataset_path, lines=True)

# Exibir as primeiras linhas do dataset
print(df.head())

# Remover linhas onde a coluna "content" está vazia
df = df[df["content"].str.strip() != ""]

# Manter apenas as colunas "uid", "title" e "content"
df = df[["uid", "title", "content"]]

# Adicionar rótulos fictícios (exemplo: 0 para "não relevante", 1 para "relevante")
# Substitua isso por rótulos reais se disponíveis
df["labels"] = np.random.randint(0, 2, size=len(df))  # Rótulos aleatórios para exemplo

# 2. Tokenização e preparação dos dados para o modelo BERT
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Função para tokenizar os títulos e retornar input_ids e attention_mask
def tokenize_titles(title):
    tokenized = tokenizer(title, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    return {
        "input_ids": tokenized["input_ids"].squeeze().numpy(),  # Converte para numpy array
        "attention_mask": tokenized["attention_mask"].squeeze().numpy(),  # Converte para numpy array
    }

# Aplicar a tokenização ao dataset
tokenized_data = df["title"].apply(tokenize_titles)

# Adicionar input_ids e attention_mask ao DataFrame
df["input_ids"] = tokenized_data.apply(lambda x: x["input_ids"])
df["attention_mask"] = tokenized_data.apply(lambda x: x["attention_mask"])

# Exibir o dataset tokenizado
print(df.head())

          uid                                              title  \
0  0000031909                        Girls Ballet Tutu Neon Pink   
1  0000032034                           Adult Ballet Tutu Yellow   
2  0000913154  The Way Things Work: An Illustrated Encycloped...   
3  0001360000                                      Mog's Kittens   
4  0001381245                              Misty of Chincoteague   

                                             content  \
0  High quality 3 layer ballet tutu. 12 inches in...   
1                                                      
2                                                      
3  Judith Kerr&#8217;s best&#8211;selling adventu...   
4                                                      

                                          target_ind  \
0  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...   
1  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...   
2                [116, 117, 118, 119, 120, 121, 122]   
3                          [14

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


           uid                        title  \
0   0000031909  Girls Ballet Tutu Neon Pink   
3   0001360000                Mog's Kittens   
7   0000031895  Girls Ballet Tutu Neon Blue   
12  000100039X                  The Prophet   
13  0001473905    Rightly Dividing the Word   

                                              content  labels  \
0   High quality 3 layer ballet tutu. 12 inches in...       0   
3   Judith Kerr&#8217;s best&#8211;selling adventu...       1   
7   Dance tutu for girls ages 2-8 years. Perfect f...       0   
12  In a distant, timeless place, a mysterious pro...       1   
13         --This text refers to thePaperbackedition.       0   

                                            input_ids  \
0   [101, 3057, 7250, 10722, 8525, 16231, 5061, 10...   
3   [101, 9587, 2290, 1005, 1055, 18401, 2015, 102...   
7   [101, 3057, 7250, 10722, 8525, 16231, 2630, 10...   
12  [101, 1996, 12168, 102, 0, 0, 0, 0, 0, 0, 0, 0...   
13  [101, 2157, 2135, 16023, 1996, 2773, 

3. Fine-Tuning do Modelo BERT

In [ ]:
!pip install sympy --upgrade

In [ ]:
!pip install transformers --upgrade

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Converter o DataFrame para o formato de Dataset da Hugging Face
dataset = Dataset.from_pandas(df)

# Dividir o dataset em treino e validação
dataset = dataset.train_test_split(test_size=0.2)

# Carregar o modelo BERT
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) # 2 classes

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/FIAP",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Função para preparar os dados para o Trainer
def preprocess_function(examples):
    return {
        "input_ids": examples["input_ids"],
        "attention_mask": examples["attention_mask"],
        "labels": examples["labels"],  # Adicionar rótulos
    }

# Aplicar a função de pré-processamento ao dataset
dataset = dataset.map(preprocess_function, batched=True)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

# Treinar o modelo
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/30741 [00:00<?, ? examples/s]

Map:   0%|          | 0/7686 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ericfontoura (ericfontoura-fiap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


4. Configuração da Integração RAG

In [ ]:
import faiss

# Criar um índice FAISS para busca de similaridade
dimension = 768  # Dimensão dos embeddings do BERT
index = faiss.IndexFlatL2(dimension)

# Gerar embeddings dos títulos usando o modelo BERT fine-tuned
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Adicionar embeddings ao índice FAISS
embeddings = np.array([generate_embeddings(title) for title in df["title"]])
index.add(embeddings)

# Função para recuperar títulos relevantes
def retrieve_relevant_titles(query, k=5):
    query_embedding = generate_embeddings(query)
    distances, indices = index.search(query_embedding, k)
    return df.iloc[indices[0]]["title"].tolist()

# Testar a recuperação
query = "melhor fone de ouvido sem fio"
relevant_titles = retrieve_relevant_titles(query)
print("Títulos relevantes:", relevant_titles)

5. Geração de Respostas

In [ ]:
# Função para gerar respostas
def generate_response(query):
    # Recuperar títulos relevantes
    relevant_titles = retrieve_relevant_titles(query)

    # Combinar a pergunta com os títulos relevantes
    context = " ".join(relevant_titles)
    prompt = f"Pergunta: {query}\nContexto: {context}\nResposta:"

    # Gerar a resposta usando o modelo fine-tuned
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_length=128)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response, relevant_titles

# Testar a geração de respostas
query = "What is the best Italian Cookbook?"
response, sources = generate_response(query)
print("Resposta:", response)
print("Fontes:", sources)

**Explicação do Código**

Fine-Tuning: O modelo BERT é ajustado para entender o contexto dos títulos de produtos da Amazon.

RAG: O sistema RAG recupera títulos relevantes com base na pergunta do usuário.

Geração de Respostas: O modelo fine-tuned gera respostas contextualizadas usando as informações recuperadas.

**Melhorias Futuras**

Aumentar o Dataset: Adicionar mais dados para melhorar a precisão do modelo.

Ajustar Hiperparâmetros: Experimentar diferentes taxas de aprendizado, épocas, etc.

Usar Modelos Mais Avançados: Experimentar com modelos como GPT-3 ou T5, se disponíveis.